In [1]:
from __future__ import division
from bokeh.plotting import figure, show, gridplot
from bokeh.io import output_notebook
from bokeh.models import Range1d
import math
import numpy as np

In [2]:
%%capture
%run 'probability.ipynb'

In [3]:
def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to Binomial(n,p)"""
    mu = p * n
    sigma = math.sqrt(n * p * (1 - p))
    return mu, sigma

In [4]:
normal_probability_below = normal_cdf

In [5]:
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

In [6]:
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

In [7]:
def normal_probability_outside(l0, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [8]:
def normal_upper_bound(probability, mu=0, sigma=1):
    return inverse_normal_cdf(probability, mu, sigma)

In [9]:
def normal_lower_bound(probability, mu=0, sigma=1):
    return inverse_normal_cdf(1 - probability, mu, sigma)

In [10]:
def normal_two_sided_bounds(probability, mu=0, sigma=1):
    tail_probability = (1 - probability) / 2
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound

In [11]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        return 2 * normal_probability_below(x, mu, sigma)

In [12]:
def count_extreme_values():
    extreme_value_count = 0
    for _ in range(100000):
        num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000))
        if num_heads >= 530 or num_heads <=470:
            extreme_value_count += 1
    extreme_value_count / 100000

In [13]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [14]:
def run_experiment():
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    num_heads = len([head for head in experiment if head])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejects = len([experiment for experiment in experiments if reject_fairness(experiment)])
num_rejects

46

In [15]:
def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

In [16]:
def a_b_test_statestic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_A - p_B) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [17]:
def B(alpha, beta):
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

In [18]:
def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

In [19]:
def make_beta_distributions():
    xs = np.arange(0,1.0001,.001)
    
    s = figure(tools="", x_range=Range1d(0,1))
    s.line(xs, [beta_pdf(x, 1, 1) for x in xs], legend="Beta(1,1)")
    s.line(xs, [beta_pdf(x, 10, 10) for x in xs], legend="Beta(10,10)", line_dash="dashdot")
    s.line(xs, [beta_pdf(x, 4, 16) for x in xs], legend="Beta(4,16)", line_dash="dotted")
    s.line(xs, [beta_pdf(x, 16, 4) for x in xs], legend="Beta(16,4)", line_dash="dashed")
    s.axis.minor_tick_line_color = None
    s.grid.grid_line_color = None
    show(s)

In [20]:
# Figure 7-1
make_beta_distributions()

In [21]:
def make_beta_different_priors():
    xs = np.arange(0,1.0001,.001)
    s = figure(tools="", x_range=Range1d(0,1))
    s.line(xs, [beta_pdf(x, 4, 8) for x in xs], legend="Beta(4,8)")
    s.line(xs, [beta_pdf(x, 23, 27) for x in xs], legend="Beta(23, 27)", line_dash="dashdot")
    s.line(xs, [beta_pdf(x, 33, 17) for x in xs], legend="Beta(33, 17)", line_dash="dotted")
    s.axis.minor_tick_line_color = None
    s.grid.grid_line_color = None
    show(s)

In [22]:
# Figure 7-2. Posteriors arising from different priors
make_beta_different_priors()